In [2]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 2 # 2aB
nH = 4
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit='bohr', verbose=4)
mol.build()

mf = scf.RHF(mol)
mf.kernel()

nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

# mycc = cc.CCD(mf,frozen=nfrozen)
# mycc.kernel()[0]

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun Oct 19 14:31:48 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000

SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpr2l4v3gn
max_memory 4000 MB (current use 169 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -1.51346449081178
  HOMO = -0.261198733616491  LUMO = 0.168676178176605
cycle= 1 E= -2.08508325783281  delta_E= -0.572  |g|= 0.0763  |ddm|= 1.36
  HOMO = -0.371230600955959  LUMO = 0.255083574566781
cycle= 2 E= -2.08837353126318  delta_E= -0.00329  |g|= 0.0206  |ddm|= 0.149
  HOMO = -0.374220926996492  LUMO = 0.262960348277124
cycle= 3 E= -2.08868615491909  delta_E= -0.000313  |g|= 0.00305  |ddm|= 0.0454
  HOMO = -0.375466774023637  LUMO = 0.263072850249689
cycle= 4 E= -2.08869238190828  delta_E= -6.23e-06  |g|= 1.06e-05  |ddm|= 0.00808
  HOMO = -0.375455607808531  LUMO = 0.263068218197618
cycle= 5 E= -2.08869238195767  delta_E= -4.94e-11  |g|= 4.83e-06  |ddm|= 1.43e-05
  HOMO = -0.37544643725064  LUMO = 0.263064839877939
Extra cycle  E= -2.08869238196961  delta_E= 

-0.07659940318513625

In [5]:
def thouless_trans(t1):
    ''' thouless transformation |psi'> = exp(t1)|psi>
        gives the transformed mo rep in the 
        original mo basis <psi_p|psi'_i>
    '''
    # t = t_ia
    # t_ia = c_ik c.T_ka
    # c_ik = <psi_i|psi'_k>
    q, r = jnp.linalg.qr(t1,mode='complete')
    u_ji = q
    u_ai = r.T
    u_occ = jnp.vstack((u_ji,u_ai))
    mo_t, _ = jnp.linalg.qr(u_occ,mode='complete')
    return mo_t

In [ ]:
# import numpy as np
# np.set_printoptions(5)
# # AFQMC/CCSD_PT energy: -1.096171 +/- 0.000554
# # AFQMC/CCSD_PT energy: -2.192221 +/- 0.001636
# # AFQMC/CCSD_PT energy: -4.384578 +/- 0.006862
# # AFQMC/CCSD_PT energy: -8.769366 +/- 0.019849
# # AFQMC/CCSD_PT energy: -17.538377 +/- 0.091299
# # AFQMC/CCSD_PT energy: -27.405295 +/- 0.246627
# nm = np.array([1,2,4,8,16,25])
# ept = np.array([-1.096171,-2.192221,-4.384578,-8.769366,-17.538377,-27.405295])
# ept_perm = ept/nm
# print('number of H2 monomers:            ', nm)
# print('AFQMC/CCSD_PT energy per monomer: ', ept_perm)

number of H2 monomers:             [ 1  2  4  8 16 25]
AFQMC/CCSD_PT energy per monomer:  [-1.09617 -1.09611 -1.09614 -1.09617 -1.09615 -1.09621]


In [4]:
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 2,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import pyscf_interface
from ad_afqmc.ccsd_pt import ccsd_pt, sample_ccsd_pt
# t1 = 5 * mycc.t1
mycc.t1 = 5*mycc.t1
ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
# pyscf_interface.prep_afqmc(mycc,chol_cut=1e-7)

#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (2, 2)
# Number of basis functions: 4
# Number of Cholesky vectors: 9
#


In [6]:
from ad_afqmc import config, mpi_jax, wavefunctions
import time
from jax import random
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

trial = wavefunctions.rhf(
    trial.norb, trial.nelec,n_batch=trial.n_batch
    )

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 4
# nelec: (2, 2)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 2
# seed: 2
# walker_type: rhf
# trial: cisd
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [ ]:
# def thouless_trans(t1):
#     q, r = np.linalg.qr(t1)
#     u_ai = r.T
#     u_ji = q
#     u_occ = np.vstack((u_ji,u_ai))
#     q, r = np.linalg.qr(u_occ)
#     # u, _, _ = np.linalg.svd(u_occ)
#     return q

In [11]:
t1, t2 = wave_data['ci1'], wave_data['ci2']
mo_t = thouless_trans(t1)
nocc = wave_data['ci1'].shape[0]
norb = trial.norb
# trial_mo = np.eye(norb)[:,:nocc]
wave_data['mo_coeff'] = np.eye(norb)[:,:nocc]
# print(t1.shape,t2.shape,trial_mo.shape)
# rot_t1 = mo_t.T.conjugate()[:,:nocc] @ t1
rot_t2 = jnp.einsum('il,jk,lakb->iajb',mo_t[:nocc,:nocc].T.conjugate(),
                   mo_t[:nocc,:nocc].T.conjugate(),t2)
wave_data['t1'] = t1
wave_data['t2'] = t2
# wave_data['rot_t1'] = rot_t1
wave_data['rot_t2'] = rot_t2
# ham_data['mo_tls'] = mo_t[:,:nocc]
wave_data['mo_tls'] = mo_t[:,:nocc]

In [9]:
import h5py
chol_file='FCIDUMP_chol'
with h5py.File(chol_file, "r") as fh5:
    [nelec, nmo, ms, nchol] = fh5["header"]
    h0 = jnp.array(fh5.get("energy_core"))
    h1 = jnp.array(fh5.get("hcore")).reshape(nmo, nmo)
    h1_mod = jnp.array(fh5.get("hcore_mod")).reshape(nmo, nmo)
    chol = jnp.array(fh5.get("chol")).reshape(-1, nmo, nmo)

ham_data['h1_mod'] = h1_mod

In [22]:
from jax import lax
### <psi|T2(h1+h2)|phi>/<psi|phi> ### 

@jax.jit
def _tls_green(walker: jax.Array, wave_data: dict) -> jax.Array:
    tls_gf = (walker.dot(jnp.linalg.inv(wave_data["mo_tls"].T.conj() @ walker))).T
    return tls_gf

@jax.jit
def _tls_walker_olp(walker, wave_data):
    ''' 
    <exp(T1)psi_0|phi>
    '''
    # o0 = trial._calc_overlap_restricted(walker, wave_data)
    # o_tls = _calc_tls_overlap_restricted(walker, ham_data)
    o_tls = jnp.linalg.det(wave_data["mo_tls"].T.conj() @ walker) ** 2
    return o_tls

@partial(jit, static_argnums=4)
def _tls_exp1(x, h1_mod, walker, wave_data, trial) -> complex:
    '''
    <psi|exp(x*h1_mod)|walker>/<psi_0|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    t2 = _tls_walker_olp(walker_1x,wave_data)
    o0 = trial._calc_overlap_restricted(walker, wave_data)
    return t2/o0

@partial(jit, static_argnums=4)
def _tls_exp2(x, chol_i, walker, wave_data, trial) -> complex:
    '''
    <psi|exp(x*h2_mod)|walker>/<psi_0|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    t2 = _tls_walker_olp(walker_2x,wave_data)
    o0 = trial._calc_overlap_restricted(walker, wave_data)
    return t2/o0

@jax.jit
def _t2_tls_walker_olp(walker, wave_data):
    ''' 
    <exp(T1) psi_0|T2|phi>
    = t_iajb <psi|ijab|phi>/<psi|phi> * <psi|phi>
    = t_iajb tls_G_iajb * <psi|phi>
    '''
    rot_t2 = wave_data['rot_t2']
    nocc = walker.shape[1]
    GF = _tls_green(walker, wave_data)
    # o0 = trial._calc_overlap_restricted(walker, wave_data)
    o_tls = _tls_walker_olp(walker, wave_data)
    t2 = 2 * jnp.einsum(
        "iajb, ia, jb", rot_t2, GF[:, nocc:], GF[:, nocc:]
    ) - jnp.einsum("iajb, ib, ja", rot_t2, GF[:, nocc:], GF[:, nocc:])
    return t2 * o_tls

@partial(jit, static_argnums=4)
def _t2_tls_exp1(x, h1_mod, walker, wave_data, trial) -> complex:
    '''
    t_iajb <psi|ijab exp(x*h1_mod)|walker>/<psi_0|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    t2 = _t2_tls_walker_olp(walker_1x,wave_data)
    o0 = trial._calc_overlap_restricted(walker, wave_data)
    return t2/o0

@partial(jit, static_argnums=4)
def _t2_tls_exp2(x, chol_i, walker, wave_data, trial) -> complex:
    '''
    t_iajb <psi|ijab exp(x*h2_mod)|walker>/<psi_0|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    t2 = _t2_tls_walker_olp(walker_2x,wave_data)
    o0 = trial._calc_overlap_restricted(walker, wave_data)
    return t2/o0

@partial(jit, static_argnums=3)
def _calc_energy_pt2(walker, ham_data, wave_data, trial):
    ''' 
    t1 = <psi|phi>/<psi_0|phi>
    t2 = <psi|T2|phi>/<psi_0|phi>
    e0 = <psi|H|phi>/<psi_0|phi>
    e1 = <psi|T2(h1+h2)|phi>/<psi_0|phi>
    '''

    eps=1e-4
    norb = trial.norb
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

    # <psi|h1+h2|phi>/<psi_0|phi>
    # one body
    # <psi|phi_1x>/<psi_0|phi>
    x = 0.0
    f1 = lambda a: _tls_exp1(a,h1_mod,walker,wave_data,trial)
    t1, d_exp1 = jvp(f1, [x], [1.0])

    # two body
    # <psi|phi_2x>/<psi_0|phi>
    def scanned_fun(carry, c):
        eps, walker, wave_data = carry
        return carry, _tls_exp2(eps,c,walker,wave_data,trial)

    _, exp2_p = lax.scan(scanned_fun, (eps, walker, wave_data), chol)
    _, exp2_0 = lax.scan(scanned_fun, (0.0, walker, wave_data), chol)
    _, exp2_m = lax.scan(scanned_fun, (-1.0 * eps, walker, wave_data), chol)
    d2_exp2 = (exp2_p - 2.0 * exp2_0 + exp2_m) / eps / eps
   
    e0 = (d_exp1 + jnp.sum(d2_exp2) / 2.0 )
    
    d_exp1 = d2_exp2 = None
    exp2_p = exp2_0 = exp2_m = None

    # <psi|T2 (h1+h2)|phi>/<psi_0|phi>
    # one body
    # t_ij^ab <psi|ijab|phi_1x>
    x = 0.0
    f1 = lambda a: _t2_tls_exp1(a,h1_mod,walker,wave_data,trial)
    t2, d_exp1 = jvp(f1, [x], [1.0])

    # two body
    # t_ij^ab <psi|ijab|phi_2x>
    def scanned_fun(carry, c):
        eps, walker, wave_data = carry
        return carry, _t2_tls_exp2(eps,c,walker,wave_data,trial)

    _, exp2_p = lax.scan(scanned_fun, (eps, walker, wave_data), chol)
    _, exp2_0 = lax.scan(scanned_fun, (0.0, walker, wave_data), chol)
    _, exp2_m = lax.scan(scanned_fun, (-1.0 * eps, walker, wave_data), chol)
    d2_exp2 = (exp2_p - 2.0 * exp2_0 + exp2_m) / eps / eps
   
    e1 = (d_exp1 + jnp.sum(d2_exp2) / 2.0 )

    return jnp.real(t1), jnp.real(t2), jnp.real(e0), jnp.real(e1)

In [23]:
from ad_afqmc.ccsd_pt import ccsd_pt2

config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1

ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

h0 = ham_data['h0']
t, e0, e1 = ccsd_pt2._ccsd_walker_energy_pt2(
    prop_data['walkers'][0],ham_data,wave_data,trial)
init_ept = e0 + e1 - t*(e0-h0)

prop_data["key"] = random.PRNGKey(seed + rank)
print('init rhf walker energy: ', prop_data['e_estimate'])
print('mf enegry: ', mf.e_tot)
print('init ccsd_pt_walker energy: ', init_ept)
print('ccsd energy: ', mycc.e_tot)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
print('ccsd energy with t1 t2: ', mf.e_tot+eccsd)
eccsd = mycc.energy(mycc.t1, 0*mycc.t2, eris)
print('ccsd energy with t1 0*t2: ', mf.e_tot+eccsd)
eccsd = mycc.energy(mycc.t1*0, mycc.t2, eris)
print('ccsd energy with 0*t1 t2: ', mf.e_tot+eccsd)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init rhf walker energy:  -2.0886923811458424
mf enegry:  -2.088692381969612
init ccsd_pt_walker energy:  -2.1652381530076146
ccsd energy:  -2.165291785154748
ccsd energy with t1 t2:  -2.165336790999147
ccsd energy with t1 0*t2:  -2.088739270083699
ccsd energy with 0*t1 t2:  -2.16528990288506


In [24]:
walker = prop_data['walkers'][0]
h0 = ham_data['h0']
t1, t2, e0, e1 = _calc_energy_pt2(walker, ham_data, wave_data, trial)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0)

-2.1653368023008768


In [18]:
t, e0, e1 = _calc_energy_pt2(
    prop_data['walkers'][0],ham_data,wave_data,trial)
print(e0 + e1 - t*(e0-h0))

-2.2136157591670838


In [18]:
trial2 = wavefunctions.ccsd_pt2_ad(trial.norb, trial.nelec, n_batch=trial.n_batch)
mo_t = trial2.thouless_trans(wave_data['t1'])
wave_data['mo_coeff'] = mo_t
t, e0, e1 = trial2._calc_energy_pt_restricted(
    prop_data['walkers'][0],ham_data,wave_data)
init_ept = e0 + e1 - t*(e0-h0)
print(init_ept)

-2.1652917893645203


In [24]:
from ad_afqmc.cisd_perturb import sample_ccsd_pt2
prop_data["n_killed_walkers"] = 0
prop_data, (blk_wt, blk_t, blk_e0, blk_e1) = \
    sample_ccsd_pt2._block_scan(prop_data,ham_data,prop,trial,wave_data,sampler)
print(blk_wt, blk_t, blk_e0, blk_e1)
print(blk_e0+blk_e1-blk_t*(blk_e0-h0))

1.9983712866932402 0.02324355211854993 -2.17346123974168 -0.1468656262740246
-2.2194468768219293


In [25]:
t, e0, e1 = ccsd_pt2._ccsd_walker_energy_pt2(
    prop_data['walkers'][0],ham_data,wave_data,trial)
print(e0 + e1 - t*(e0-h0))
t, e0, e1 = _calc_energy_pt2(
    prop_data['walkers'][0],ham_data,wave_data,trial)
print(e0 + e1 - t*(e0-h0))

-2.2034847267664386
-2.2034847360714407
